In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from transformers import AutoModel, AutoTokenizer

In [ ]:
df = pd.read_csv('data.csv')

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_texts, test_texts, train_labels, test_labels = train_test_split(df['free_text'], df['label_id'], test_size=0.2, random_state=42)

# Tải mô hình PhoBERT và tokenizer
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
def text_to_phobert_embedding(text, max_length=256):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    if 'input_ids' not in tokens or len(tokens['input_ids'][0]) == 0:
        return np.zeros((1, phobert_model.config.hidden_size))

    with torch.no_grad():
        outputs = phobert_model(**tokens)

    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

In [ ]:
train_texts = train_texts.astype(str).tolist()

# Chuyển đổi tất cả văn bản thành biểu diễn của PhoBERT
train_embeddings = np.array([text_to_phobert_embedding(text) for text in train_texts])

test_texts = test_texts.astype(str).tolist()

# Chuyển đổi tất cả văn bản thành biểu diễn của PhoBERT
test_embeddings = np.array([text_to_phobert_embedding(text) for text in test_texts])

In [ ]:
# Xây dựng mô hình
model = Sequential()
model.add(LSTM(32, input_shape=(train_embeddings.shape[1], train_embeddings.shape[2])))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Biên soạn và đào tạo mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_embeddings, train_labels, epochs=10, batch_size=1)

Epoch 1/10
8844/8844 [==============================] - 51s 6ms/step - loss: 0.4818 - accuracy: 0.7685
Epoch 2/10
8844/8844 [==============================] - 49s 6ms/step - loss: 0.4335 - accuracy: 0.7983
Epoch 3/10
8844/8844 [==============================] - 48s 5ms/step - loss: 0.4057 - accuracy: 0.8087
Epoch 4/10
8844/8844 [==============================] - 48s 5ms/step - loss: 0.3816 - accuracy: 0.8256
Epoch 5/10
8844/8844 [==============================] - 50s 6ms/step - loss: 0.3599 - accuracy: 0.8384
Epoch 6/10
8844/8844 [==============================] - 48s 5ms/step - loss: 0.3343 - accuracy: 0.8541
Epoch 7/10
8844/8844 [==============================] - 47s 5ms/step - loss: 0.3131 - accuracy: 0.8630
Epoch 8/10
8844/8844 [==============================] - 50s 6ms/step - loss: 0.2848 - accuracy: 0.8788
Epoch 9/10
8844/8844 [==============================] - 48s 5ms/step - loss: 0.2540 - accuracy: 0.8938
Epoch 10/10
8844/8844 [==============================] - 49s 6ms/step - l

In [ ]:
# Kiểm thử mô hình với dữ liệu kiểm tra
predictions = model.predict(test_embeddings)
predictions_binary = np.round(predictions).flatten()

70/70 [==============================] - 1s 3ms/step


In [ ]:
# Đánh giá mô hình
accuracy = np.sum(predictions_binary == test_labels) / len(test_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 79.38%


In [ ]:
# Chuẩn bị câu cần kiểm tra
sample_text = "con đuỹ"

# Chuyển đổi câu thành biểu diễn của PhoBERT
sample_tokens = tokenizer(sample_text, return_tensors="pt", truncation=True, max_length=256)
with torch.no_grad():
    sample_outputs = phobert_model(**sample_tokens)

# Lấy vector biểu diễn của [CLS] token
sample_embedding = sample_outputs.last_hidden_state[:, 0, :].numpy()

# Dự đoán bằng mô hình đã được đào tạo
sample_prediction = model.predict(np.array([sample_embedding]))

# In kết quả
print(f"Câu: {sample_text}")
print(f"Dự đoán: {'Tục' if sample_prediction > 0.5 else 'Không tục'}")

In [ ]:
# Lưu mô hình
model.save("vietnam_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
